In [4]:
import camelot
import pandas as pd

# Updated Process

## 1. Create empty dataframe with proper column names

In [5]:
# This is where data will be added after cleaning. The final dataframe for export

In [6]:
df = pd.DataFrame(columns = [
    "Species",
    "BE", 
    "Bestand", 
    "Trend_lang",
    "Trend_kurz",
    "RF",
    "BB",
    "D",
    "GS",
    "GfU",
    "Common_name"
])
df

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name


In [7]:
df.shape

(0, 11)

## 2. Setup mechanism to store accuracy report : QA step

In [8]:
accuracy = []

## 3. Dictionary of dataframes on the go

In [9]:
# Dictionary to collect dataframes created from each table
d = {}

## 4. Adding the list of pages to be parsed

In [10]:
pages = []
i = 7
while i < 8:
    pages.append(i)
    i = i + 1
print(pages)

[7]


## 5. Parsing through the pages

In [11]:
for i in pages:
    # print page number
    print (i)
    # create dataframe with page number
    d[i] = pd.DataFrame()
    #read the page
    tables = camelot.read_pdf('rote_liste_reptilien_kuehnel_et_al.pdf', 
                              pages= str(i), 
                              flavor='stream', edge_tol=1000, row_tol=10)
    #tables = camelot.read_pdf('18_netzfl_print.pdf', pages= str(i), flavor='lattice')
    # add the accuracy report
    accuracy.append(tables[0].parsing_report)
    # assign the first table to a dataframe
    d[i] = tables[0].df
    print(d[i].shape)
# Print Done once all pages are parsed
print ("Done")

7
(17, 11)
Done


In [12]:
# Check accuracy reports for the process
accuracy

[{'accuracy': 99.74, 'whitespace': 49.2, 'order': 1, 'page': 7}]

# Data cleanup steps

In [13]:
# Check whether all dataframes are of the same size
for key in d:
    print(key)
    print(d[key].shape)

7
(17, 11)


In [14]:
# Removing empty dataframes
# del d[64]
# del d[74]

In [15]:
# Creates a sorted dictionary (sorted by key)
from collections import OrderedDict
d2 = OrderedDict(sorted(d.items()))

In [16]:
# Check whether the dictionary is sorted
for key in d2:
    print(key)

7


In [17]:
# Rename all the dataframes in the sorted dictionary
for key in d2:
    print(key)
    d2[key].columns = ["Species",
    "BE", 
    "Bestand", 
    "Trend_lang",
    "Trend_kurz",
    "RF",
    "BB",
    "D",
    "GS",
    "GfU",
    "Common_name"]
print("Rename completed for concat")

7
Rename completed for concat


In [18]:
# Concatanate all the non-empty dataframes in the sorted dictionary into the empty df
for key in d2:
    df = pd.concat([df, d2[key]], axis = 0)
df

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
0,Wissenschaftlicher Name,BE,Bestand,Trend,Trend,RF,BB,D,GS,GfU,Deutscher Name
1,,,,lang,kurz,,,,,,
2,Schildkröten (Testudines),,,,,,,,,,
3,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte
4,,,,,,,,,,"14c, 14i, 14l",
5,Echsen (Sauria),,,,,,,,,,
6,"Anguis fragilis (LINNAEUS, 1758)",V,mh,<,((cid:114)),=,**,*,§,"2a, 3a, 8e, 9a, 12b, 14i","Blindschleiche,"
7,,,,,,,,,,,Westliche Blindschleiche
8,"Lacerta agilis (LINNAEUS, 1758)",V,h,<<,((cid:114)),-,3,V,§§,"1a, 1c, 2a, 4b, 6d, 7a, 8a, 8d,",Zauneidechse
9,,,,,,,,,,"8e, 12a, 12b, 14i",


In [19]:
# Definition of extinction
df2 = df.loc[df["BE"] == "0"]
df2.head()

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
3,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte


In [20]:
df2.reset_index(inplace = True)
df2.head()

,index,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
0,3,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte


In [21]:
df2 = df2.drop(['index'], axis=1)
df2.head()

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
0,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte


In [22]:
df3 = df.loc[df["Bestand"] == "ex"]
df3.reset_index(inplace = True)
df3 = df3.drop(['index'], axis=1)
df3.head()

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
0,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte


In [23]:
df2.describe()

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
count,1,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,1
top,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte
freq,1,1,1,1,1,1,1,1,1,1,1


In [24]:
df3.describe()

,Species,BE,Bestand,Trend_lang,Trend_kurz,RF,BB,D,GS,GfU,Common_name
count,1,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,1
top,"Emys orbicularis (LINNAEUS, 1758)",0,ex,,,,1,1,"§§, II, IV","4a, 5a, 5b, 10c, 12b , 13a, 14b,",Europäische Sumpfschildkröte
freq,1,1,1,1,1,1,1,1,1,1,1


In [26]:
# export the dataframe as CSV
df2.to_csv('Reptilia.csv', index=False)